## Задание 5

### Постановка задачи

Для иерархической структуры, представленной на рисунке, определите приоритет провайдера, выполнив иерархический синтез.

<img src="files/pic6.png">

Матрица сравнения критериев относительно цели имеет вид:
$$
A=\left(\begin{array}{ccccc}
1 & 4 & 6 & 2 & 7 \\
1/4 & 1 & 3 & 4 & 2 \\
1/6 & 1/3 & 1 & 2 & 1 \\
1/2 & 1/4 & 1/2 & 1 & 1/3 \\
1/7 & 1/2 & 1 & 3 & 1
\end{array}\right).
$$

Матрицы сравнения альтернатив относительно критериев считаются заданными.

### Ход решения

Иерархический синтез используется для общего ранжирования альтернатив относительно цели, т.е. для подсчета количественной оценки качества альтернатив. Действовать будем согласно алгоритму.

**Шаг 1.** Для каждого элемента иерархии построить матрицы парных сравнений элементов иерархии следующего уровня. На первом уровне иерархии будем использовать матрицу сравнения относительно удовлетворения провайдером [УП].

$$
[УП] = \left(\begin{array}{ccccccc}
 & & \underline{Т} & \underline{С} & \underline{Д} & \underline{О} & \underline{У} \\
Т & | & 1 & 4 & 6 & 2 & 7 \\
С & | & 1/4 & 1 & 3 & 4 & 2 \\
Д & | & 1/6 & 1/3 & 1 & 2 & 1 \\
О & | & 1/2 & 1/4 & 1/2 & 1 & 1/3 \\
У & | & 1/7 & 1/2 & 1 & 3 & 1 
\end{array}\right).
$$

In [27]:
A = [1 4 6 2 7; 1/4 1 3 4 2; 1/6 1/3 1 2 1; 1/2 1/4 1/2 1 1/3; 1/7 1/2 1 3 1]

5×5 Array{Float64,2}:
 1.0       4.0       6.0  2.0  7.0     
 0.25      1.0       3.0  4.0  2.0     
 0.166667  0.333333  1.0  2.0  1.0     
 0.5       0.25      0.5  1.0  0.333333
 0.142857  0.5       1.0  3.0  1.0     

Введем функцию вычисления приближенных значений максимального собственного числа и главного собственного вектора.

In [28]:
function eigvalvecapprox(A)
    eps = 1e-9
    e = ones(size(A)[1])
    num = A * e
    denom = e' * num
    prev = num / denom
    num = A * num
    denom = e' * num
    next = num / denom
    while true
        prev = next
        num = A * num
        denom = e' * num
        next = num / denom
        result = abs(next - prev) .> eps
        done = true
        for r in result
            if !r
                done = false
                break
            end
        end
        if done
            break
        end
    end
    Wa = next
    lmax = (e' * A * Wa) / (e' * Wa)
    return lmax[1], Wa
end

eigvalvecapprox (generic function with 1 method)

Вычислим приближенные значения максимального собственного числа и главного собственного вектора для матрицы $A$.

In [29]:
val, vec = eigvalvecapprox(A)

(5.643273525681777,
[0.505324; 0.205149; … ; 0.0867938; 0.110393])

Введем функцию для вычисления индекса однородности (индекса согласованности)
$$
I_S=\frac{\lambda_{max}-n}{n-1}.
$$

In [30]:
function ids(lmax, n)
    return (lmax - n) / (n - 1)
end

ids (generic function with 1 method)

Вычислим индекс однородности для матрицы $A$.

In [31]:
Is = ids(val, size(A)[1])

0.16081838142044425

Также, потребуется функция для вычисления отношения однородности (отношения согласованности)
$$
R_S=\frac{I_S}{\hat{I_S}}.
$$

In [32]:
function rs(idx, n)
    mean = 0
    if n == 3
        mean = 0.58
    elseif n == 4
        mean = 0.9
    elseif n == 5
        mean = 1.12
    end
    return idx / mean
end

rs (generic function with 1 method)

Вычислим отношение однородности для матрицы $A$.

In [33]:
Rs = rs(Is, size(A)[1])

0.14358784055396806

In [34]:
As = []
push!(As, [1 4 6 8; 1/4 1 3 2; 1/6 1/3 1 3; 1/8 1/2 1/3 1])
push!(As, [1 3 6 9; 1/3 1 1/4 2; 1/6 4 1 3; 1/9 1/2 1/3 1])
push!(As, [1 8 3 2; 1/8 1 3 2; 1/3 1/3 1 1; 1/2 1/2 1 1])
push!(As, [1 4 6 1; 1/4 1 3 2; 1/6 1/3 1 3; 1 1/2 1/3 1])
push!(As, [1 3 2 8; 1/3 1 1/4 1/2; 1/2 1/4 1 3; 1/8 2 1/3 1])
As

5-element Array{Any,1}:
 [1.0 4.0 6.0 8.0; 0.25 1.0 3.0 2.0; 0.166667 0.333333 1.0 3.0; 0.125 0.5 0.333333 1.0]   
 [1.0 3.0 6.0 9.0; 0.333333 1.0 0.25 2.0; 0.166667 4.0 1.0 3.0; 0.111111 0.5 0.333333 1.0]
 [1.0 8.0 3.0 2.0; 0.125 1.0 3.0 2.0; 0.333333 0.333333 1.0 1.0; 0.5 0.5 1.0 1.0]         
 [1.0 4.0 6.0 1.0; 0.25 1.0 3.0 2.0; 0.166667 0.333333 1.0 3.0; 1.0 0.5 0.333333 1.0]     
 [1.0 3.0 2.0 8.0; 0.333333 1.0 0.25 0.5; 0.5 0.25 1.0 3.0; 0.125 2.0 0.333333 1.0]       

Для каждой из матриц найти максимальные собственные значения (они потребуются для оценки однородности суждений) и главные собственные векторы, элементы которых равны приоритетам соответствующих элементов следующего уровня иерархии. Для этого воспользуемся алгоритмом вычисления точного значения максимального собственного числа и главного собственного вектора из задания 1, который оформим в виде функции.

In [35]:
function eigvalvecaccurate(A)
    a = Complex(A[1, 2])
    b = Complex(A[1, 3])
    c = Complex(A[1, 4])
    d = Complex(A[2, 3])
    e = Complex(A[2, 4])
    f = Complex(A[3, 4])
    B = (d*f/e+e/d/f)+(a*e/c+c/a/e)+(b*f/c+c/b/f)+(a*d/b+b/a/d)
    C = 3-(a*d*f/c+c/a/d/f)-(a*e/b/f+b*f/a/e)-(c*d/a/e)-(c*d/b/e+b*e/c/d)
    x = B^2/2+8C-8
    y = (4(C+3)/3)^3
    r = (x+sqrt(y+x^2))^(1/3) + (x-sqrt(y+x^2))^(1/3)
    lmax = (2+sqrt(r+4))/2 + sqrt((8-r)/4) + B/(2sqrt(r+4))
    Q = (lmax-1)^3 + (c+f+e)*(lmax-1)^2 + ((a*e-3)+(b+d)f+c/a+c/b+e/d)*(lmax-1) + (a*d*f-c-e-f+(b*e/d+b*f/a)+(c*d+a*e-a*d)/b+(c-b)/a/d)
    first = c*(lmax-1)^2 + (a*e+b*f)*(lmax-1) + (a*d*f+b*e/d-c)
    second = e*(lmax-1)^2 + (d*f+c/a)*(lmax-1) + (b*f/a+c*d/b-e)
    third = f*(lmax-1)^2 + (e/d+c/b)*(lmax-1) + (c/a/d+a*e/b-f)
    fourth = (lmax-1)^3 - 3*(lmax-1) - (b/a/d+a*d/b)
    accurate = [first; second; third; fourth]
    accurate /= Q
    return lmax, accurate
end

eigvalvecaccurate (generic function with 1 method)

In [36]:
Vals = []
Vecs = []

for A in As
    maxval, mainvec = eigvalvecaccurate(A)
    push!(Vals, maxval)
    push!(Vecs, mainvec)
end

In [37]:
Vals

5-element Array{Any,1}:
 5.43324+0.0im     
 5.70915+0.0im     
  5.6795-0.347267im
 7.08261-1.17543im 
 5.50552+0.0im     

In [38]:
Vecs

5-element Array{Any,1}:
 Complex{Float64}[0.566791+0.0im,0.17604+0.0im,0.124394+0.0im,0.132774+0.0im]                           
 Complex{Float64}[0.567362+0.0im,0.106442+0.0im,0.194434+0.0im,0.131762+0.0im]                          
 Complex{Float64}[0.475174+0.0270636im,0.180733+0.00219074im,0.0989208+0.0026145im,0.245172-0.0318688im]
 Complex{Float64}[0.333071+0.0606442im,0.194139+0.0149347im,0.169576-0.00135721im,0.303214-0.0742217im] 
 Complex{Float64}[0.493833+0.0im,0.0712249+0.0im,0.244705+0.0im,0.190237+0.0im]                         

Построим результирующую матрицу $A$ со средним геометрическим оценок.

In [39]:
using Stats
A = map((x1, x2, x3, x4, x5) -> geomean([x1, x2, x3, x4, x5]), As[1], As[2], As[3], As[4], As[5])

4×4 Array{Float64,2}:
 1.0       4.09535   4.19296   4.09535
 0.24418   1.0       1.11032   1.51572
 0.238495  0.517282  1.0       2.40822
 0.24418   0.659754  0.415244  1.0    